## 985 211 list

In [1]:
from bs4 import BeautifulSoup
import requests

In [14]:
import contextlib

In [42]:
import pickle

In [2]:
webpage = "https://www.applyto.com/node/99"

In [3]:
contents = requests.get(webpage)

In [6]:
soup = BeautifulSoup(contents.text)

In [24]:
school_list = []
for row in soup.find_all("td"):
    if not row.find('p'):
        with contextlib.suppress(KeyError):
            school_list.append(row.attrs['data-sheets-value'].split(":")[-1].replace('"','').replace('}', ''))

In [26]:
school_list = [item for item in school_list if '所' not in item]

In [33]:
school_211 = school_list[1:school_list.index('“985”工程大学名单')]
school_985 = school_list[school_list.index('“985”工程大学名单') + 1:]

In [38]:
school_211 = [item.split("(")[0] for item in school_211]

In [43]:
with open('211.pickle', 'wb') as f:
    pickle.dump(school_211, f)

In [44]:
with open('985.pickle', 'wb') as f:
    pickle.dump(school_985, f)

In [50]:
wp = "https://www.compassedu.hk/class_t6p1"

In [52]:
import urllib3
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS = 'ALL:@SECLEVEL=1'

In [91]:
import time

In [109]:
from tqdm import tqdm

In [189]:
def scrape_zhinanzhe_majors(wp, starting_page, total_pages):
    majors = []
    for i in tqdm(range(starting_page, total_pages + 1)):
        try:
            _wp = wp + str(i)
            # print(wp)
            contents = requests.get(_wp)
            soup = BeautifulSoup(contents.text)
            s = soup.find_all('section')[-1] # search results that matter to us
            items = s.find_all('a')
            # print(len(items))
            for item in items:
                major_ch_name = item.find('div', attrs = 'cname').contents[0]
                major_en_name = item.find('div', attrs='ename line-1').contents[0]
                major_url = item.attrs['href']
                major_ranking = item.find('div', attrs='sort').contents[0]
                all_tags = item.find_all('div', attrs = 'tag-item')
                all_tags_content = [c.contents[0] for c in all_tags]
                majors.append(Major(major_en_name, major_ch_name, major_url, major_ranking, all_tags_content))
            time.sleep(5)
        except Exception as e:
            print(e)
    return majors

In [190]:
res = scrape_zhinanzhe_majors("https://www.compassedu.hk/class_p", 1, 286)

 26%|██████████▏                             | 73/286 [11:35<1:35:28, 26.90s/it]

list index out of range


100%|█████████████████████████████████████████| 286/286 [43:15<00:00,  9.07s/it]


In [197]:
import sys
sys.setrecursionlimit(10000)

In [194]:
import pickle

In [198]:
with open('majors.pkl', 'wb') as f:
    pickle.dump(res, f)

In [202]:
len(res)

8524

In [205]:
import pandas as pd

In [208]:
major_df = pd.DataFrame(columns=['en_name', 'ch_name', 'url', 'ranking', 'area'])

In [217]:
res_list = [(row.en_name, row.ch_name, row.url, row.ranking, ' '.join(row.tag)) for row in res]

In [220]:
major_df = pd.DataFrame(res_list, columns=['en_name', 'ch_name', 'url', 'ranking', 'area'])

In [222]:
major_df.to_csv("专业名称及排名.csv", index=None)

In [ ]:
with open('mypickle.pickle', 'wb') as f:
    pickle.dump(some_obj, f)

In [87]:
from dataclasses import dataclass

In [105]:
@dataclass
class Major:
    en_name: str
    ch_name: str
    url: str
    ranking: str
    tag: list

# Scrape each major

In [224]:
wp = "https://www.compassedu.hk/majr_1747"

In [225]:
contents = requests.get(wp)

In [226]:
soup = BeautifulSoup(contents.text)

In [250]:
@dataclass
class MajorDetails:
    area: str
    area_ranking: int
    entrance_time: str
    project_length: str
    tuition: str
    url: str

In [ ]:
#major_details = []

In [260]:
base_url = 'https://www.compassedu.hk'

for i in tqdm(range(6519, len(major_df))):
    try:
        url = base_url + major_df.iloc[i]['url']
        contents = requests.get(url)
        soup = BeautifulSoup(contents.text)
        for item in soup.find_all('div', attrs='project-item'):
            if item.find_next('div').contents[0] == '专业方向':
                area = item.find_all('div')[-1].contents[0]
            if item.find_next('div').contents[0] == '专业排名':
                area_ranking = item.find_all('div')[-1].contents[0]
            if item.find_next('div').contents[0] == '入学时间':
                entrance_time = item.find_all('div')[-1].contents[0]
            if item.find_next('div').contents[0] == '项目时长':
                project_length = item.find_all('div')[-1].contents[0]
            if item.find_next('div').contents[0] == '项目学费':
                tuition = item.find_all('div')[-1].contents[0]
        major_detail = MajorDetails(area, area_ranking, entrance_time, project_length, tuition, major_df.iloc[i]['url'])
        major_details.append(major_detail)
        time.sleep(0.5)
    except Exception as e:
        print(e)

100%|█████████████████████████████████████| 2005/2005 [1:36:36<00:00,  2.89s/it]


In [261]:
len(major_details)

8524

In [262]:
len(major_df)

8524

In [263]:
res_list = [(row.area, row.area_ranking, row.entrance_time, row.project_length, 
             row.tuition, row.url) for row in major_details]

In [265]:
major_detail_df = pd.DataFrame(res_list, columns=['area_1', 'area_ranking', 'entrance_time', 'project_length', 
                                                  'tuition', 'url'])

In [479]:
df_all = pd.concat([major_df, major_detail_df], axis=1)

In [473]:
def map_region(tuition):
    if '美' in tuition:
        return 'US'
    if '英' in tuition:
        return 'Britain'
    if '澳' in tuition:
        return 'Australia'
    if '港' in tuition:
        return 'HongKong'
    if '新' in tuition:
        return 'Singapore'
    if '欧' in tuition:
        return 'Europe'
    else:
        return 'Unknown'

In [482]:
df_all['region'] = df_all['tuition'].apply(map_region)

In [484]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8524 entries, 0 to 8523
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   en_name         8524 non-null   object
 1   ch_name         8524 non-null   object
 2   url             8524 non-null   object
 3   ranking         8524 non-null   object
 4   area            8524 non-null   object
 5   tofel           8346 non-null   object
 6   ielts           8447 non-null   object
 7   gmat            817 non-null    object
 8   gre             802 non-null    object
 9   tofel_sub       8346 non-null   object
 10  ielts_sub       8447 non-null   object
 11  gmat_sub        817 non-null    object
 12  gre_sub         802 non-null    object
 13  curriculum      8510 non-null   object
 14  area_1          8524 non-null   object
 15  area_ranking    8524 non-null   object
 16  entrance_time   8524 non-null   object
 17  project_length  8524 non-null   object
 18  tuition 

In [299]:
df_all[['en_name', 'ch_name', 'ranking', 'area', 'area_1', 'area_ranking', 
        'entrance_time', 'project_length', 'tuition', 'region']].to_csv('指南者专业信息.csv', index=None)

In [ ]:
''.join(i for i in myStr if not i.isdigit())

In [284]:
k = df_all['tuition'].apply(lambda x: x.split('/')[0].strip().replace('.','').replace('-', ''))

In [285]:
k

0        80718美元
1        82368美元
2        55510美元
3        55510美元
4        55510美元
          ...   
8519     96480港币
8520    170700港币
8521    224640港币
8522    115200港币
8523     99840港币
Name: tuition, Length: 8524, dtype: object

In [286]:
k = k.apply(lambda x: ''.join(i for i in x if not i.isdigit())) 

In [287]:
k.unique()

array(['美元', '英镑', ' 美元', '人民币', '新币', '欧元', '', '港币', ',美元', '澳元', '澳币',
       '   美元', '万美元'], dtype=object)

In [246]:
for item in soup.find_all('div', attrs='project-item'):
    if item.find_next('div').contents[0] == '专业方向':
        area = item.find_all('div')[-1].contents[0]
    if item.find_next('div').contents[0] == '专业排名':
        area_ranking = item.find_all('div')[-1].contents[0]
    if item.find_next('div').contents[0] == '入学时间':
        entrance_time = item.find_all('div')[-1].contents[0]
    if item.find_next('div').contents[0] == '项目时长':
        project_length = item.find_all('div')[-1].contents[0]
    if item.find_next('div').contents[0] == '项目学费':
        tuition = item.find_all('div')[-1].contents[0]
major_detail = MajorDetails(area, area_ranking, entrance_time, project_length, tuition)

In [247]:
major_detail

MajorDetails(area='建筑', area_ranking='3', entrance_time='9月', project_length='1.5年', tuition='146667港币/年')

In [241]:
k.find_next('div')

<div class="label">专业方向</div>

In [245]:
soup.find_all('div', attrs='project-item')

[<div class="project-item">
 <div class="label">专业方向</div>
 <div class="value">建筑</div>
 </div>,
 <div class="project-item">
 <div class="label">专业排名</div>
 <div class="value">3</div>
 </div>,
 <div class="project-item">
 <div class="label">入学时间</div>
 <div class="value">9月</div>
 </div>,
 <div class="project-item">
 <div class="label">项目时长</div>
 <div class="value">1.5年</div>
 </div>,
 <div class="project-item">
 <div class="label">项目学费</div>
 <div class="value">146667港币/年</div>
 </div>]

# Scrape each major for more details


In [350]:
wp = "https://www.compassedu.hk/majr_50007"
contents = requests.get(wp)
soup = BeautifulSoup(contents.text)

In [343]:
@dataclass
class EnglishRequirement:
    tofel: str
    ielts: str
    gmat: str
    gre: str
    tofel_sub: str
    ielts_sub: str
    gmat_sub: str
    gre_sub: str

In [352]:
def get_lang_requirements(soup):
    lang_requirements = soup.find_all('div', attrs='detail__module__language')[0]
    toefl_total, toefl_sub, ielts_total, ielts_sub, gre_total, gre_sub, gmat_total, gmat_sub = \
    None, None, None, None, None, None, None, None
    try:
        details = lang_requirements.find_all('div', attrs = 'row-item')
        for detail in details:
            if detail.find_next('span').contents[0] == '托福':
                toefl_total = detail.find_all('span')[-2].contents[0]
                toefl_sub = detail.find_all('span')[-1].contents[0]
            if detail.find_next('span').contents[0] == '雅思':
                ielts_total = detail.find_all('span')[-2].contents[0]
                ielts_sub = detail.find_all('span')[-1].contents[0]
            if detail.find_next('span').contents[0].lower() == 'gmat':
                gmat_total = detail.find_all('span')[-2].contents[0]
                gmat_sub = detail.find_all('span')[-1].contents[0]
            if detail.find_next('span').contents[0].lower() == 'gre':
                gre_total = detail.find_all('span')[-2].contents[0]
                gre_sub = detail.find_all('span')[-1].contents[0]
    except:
        pass
    return EnglishRequirement(toefl_total, ielts_total, gmat_total, gre_total, toefl_sub, ielts_sub, gmat_sub, gre_sub)

In [378]:
def get_classes(soup):
    classes = ['']
    try:
        t = soup.find_all('div', attrs='curriculum-list')[0]
        classes = [item.find_all('span')[1].contents[0] for item in t.find_all('div', attrs='row-item')][1:]
    except:
        pass
    return ', '.join(classes)

In [385]:
classes_dict = {}
eng_requirement_dict = {}

In [389]:
base_url = 'https://www.compassedu.hk'

for i in tqdm(range(0, len(major_df))):
    try:
        url = base_url + major_df.iloc[i]['url']
        contents = requests.get(url)
        soup = BeautifulSoup(contents.text)
        classes_dict[major_df.iloc[i]['url']] = get_classes(soup)
        eng_requirement_dict[major_df.iloc[i]['url']] = get_lang_requirements(soup)
    except:
        pass
    time.sleep(0.5)

100%|████████████████████████████████████| 8524/8524 [42:16:33<00:00, 17.85s/it]


In [393]:
with open('eng_requirement_dict.pickle', 'wb') as f:
    pickle.dump(eng_requirement_dict, f)

In [398]:
with open('classes_dict.pickle', 'wb') as f:
    pickle.dump(classes_dict, f)

In [399]:
len(eng_requirement_dict)

8451

In [400]:
len(classes_dict)

8510

In [401]:
len(major_df)

8524

In [402]:
major_df.head()

,en_name,ch_name,url,ranking,area
0,Master of Finance,麻省理工学院金融硕士,/majr_65569,QS排名：1,商科 金融
1,Master of Business Analytics,麻省理工学院商业分析硕士,/majr_65504,QS排名：1,商科 商业分析
2,Master of Science in Mechanical Engineering (S...,麻省理工学院机械工程理学硕士,/majr_71808,QS排名：1,工科 机械工程
3,Master of Science in Materials Science and Eng...,麻省理工学院材料科学与工程理学硕士,/majr_71870,QS排名：1,工科 材料
4,MS in Chemical Engineering Practice,麻省理工学院化学工程实践理学硕士,/majr_71771,QS排名：1,工科 化工


In [407]:
list(eng_requirement_dict.values())[0]

EnglishRequirement(tofel='/', ielts='/', gmat='/', gre='/', tofel_sub='/', ielts_sub='/', gmat_sub='/', gre_sub='/')

In [408]:
eng_requirement_list = list(eng_requirement_dict.values())

In [413]:
eng_requirement_list = [[k.tofel, k.ielts, k.gmat, k.gre, k.tofel_sub, k.ielts_sub, k.gmat_sub, k.gre_sub] for k in eng_requirement_list]

In [415]:
eng_requirement_df = pd.DataFrame(eng_requirement_list, columns=['tofel', 'ielts', 'gmat', 'gre', 
                                                                 'tofel_sub', 'ielts_sub', 'gmat_sub', 'gre_sub'])

In [421]:
eng_requirement_df['url'] = list(eng_requirement_dict.keys())

In [425]:
# eng_requirement_df = eng_requirement_df.drop(['major_url'], axis=1)

In [439]:
classes_df = pd.DataFrame([classes_dict.keys(), classes_dict.values()])

In [441]:
classes_df = classes_df.T

In [446]:
classes_df.columns = ['url', 'curriculum']

In [485]:
df_all = df_all.loc[:,~df_all.columns.duplicated()].copy()

In [487]:
df_all.to_csv("指南者专业信息全.csv", index=None)

# Scrape applicant examples


In [488]:
wp = 'https://www.compassedu.hk/offer_c7t6p1'
contents = requests.get(wp)
soup = BeautifulSoup(contents.text)

In [531]:
region_dict = {'英国': 'c7', '美国': 'c12', '香港': 'c11', '澳大利亚': 'c9', '新加坡':'c10'}
major_dict = {'商科': 't6', '社科':'t4', '工科': 't1', '理科': 't5'}

In [529]:
def scrape_applicants(soup):
    result = soup.find_all('div', attrs='result-list')[0]
    schools = result.find_all('div', attrs='info')
    applicant_details = result.find_all('div', attrs='detail')
    try:
        assert len(schools) == len(applicant_details)
    except AssertionError:
        print("bad form of input data")
        return None
    
    school_list = []
    for school in schools:
        school_list.append(school.find('div', attrs='title line-2').contents[0].replace('研究生offer一枚', ''))
    
    applicant_detail_list = []
    for detail in applicant_details:
        items = detail.find_all('div', attrs='text line-1')
        applicant_detail_list.append(', '.join([item.contents[0] for item in items[1:]]))
    
    try:
        assert len(school_list) == len(applicant_detail_list)
    except AssertionError:
        print ("bad scraping")
        return None
    
    return school_list, applicant_detail_list

In [550]:
all_applicant_details = {}

In [551]:

for region in region_dict.keys():
    for area in major_dict.keys():
        all_applicant_details[f'{region}_{area}'] = ([], [])
        

In [553]:
for region in region_dict.keys():
    for area in major_dict.keys():
        for i in tqdm(range(55)):
            try:
                wp = f'https://www.compassedu.hk/offer_{region_dict[region]}{major_dict[area]}p{i}'
                contents = requests.get(wp)
                soup = BeautifulSoup(contents.text)
                results = scrape_applicants(soup)
                all_applicant_details[f'{region}_{area}'][0].extend(results[0])
                all_applicant_details[f'{region}_{area}'][1].extend(results[1])
            except Exception as e:
                print(e)
            time.sleep(0.5)

  0%|                                                    | 0/55 [00:00<?, ?it/s]

list index out of range


  0%|                                                    | 0/55 [00:00<?, ?it/s]

list index out of range


  0%|                                                    | 0/55 [00:00<?, ?it/s]

...

  0%|                                                    | 0/55 [00:00<?, ?it/s]

list index out of range


  0%|                                                    | 0/55 [00:00<?, ?it/s]

list index out of range


100%|███████████████████████████████████████████| 55/55 [02:45<00:00,  3.00s/it]


In [555]:
all_applicant_details.keys()

dict_keys(['英国_商科', '英国_社科', '英国_工科', '英国_理科', '美国_商科', '美国_社科', '美国_工科', '美国_理科', '香港_商科', '香港_社科', '香港_工科', '香港_理科', '澳大利亚_商科', '澳大利亚_社科', '澳大利亚_工科', '澳大利亚_理科', '新加坡_商科', '新加坡_社科', '新加坡_工科', '新加坡_理科'])

In [557]:
pd.DataFrame(all_applicant_details['英国_商科']).T

,0,1
0,卡迪夫大学国际经济学、银行与金融理学硕士,"郑州航空工业管理学院, 经济学, 三年以上工作经验，GPA76.01，无"
1,格拉斯哥大学信息管理与保存理学硕士,"华东师范大学, 信息管理与信息系统, 已毕业，GPA81.52，无"
2,剑桥大学经济学哲学硕士,"中央财经大学, 国际经济与贸易（国际贸易/金融风险管理方向）, 大四，GPA91.72，雅思..."
3,伦敦大学国王学院金融学（资产定价）理学硕士,"中国海洋大学, 金融学, 大四，GPA97.93，无"
4,南安普顿大学会计与管理理学硕士,"湖南农业大学, 金融学, 大四，GPA81.2，暂无"
...,...,...
1577,华威大学供应链与物流管理理学硕士,"华中科技大学, 电气工程及其自动化, 大四，GPA 78分，雅思6分"
1578,雷丁大学投资管理学理学硕士,"东南大学, 市政工程, 大四，GPA 81分，雅思6分"
1579,杜伦大学金融学理学硕士,"南京理工大学, 金融学, 大四，GPA78分，雅思6分"
1580,兰卡斯特大学电子商务与革新理学硕士,"南京信息工程大学, 金融工程, 大四，GPA 76分，雅思6分"


In [559]:
for key in all_applicant_details.keys():
    df = pd.DataFrame(all_applicant_details[key]).T
    df.columns =['school', 'applicant']
    df.to_csv(f'{key}_applicants.csv', index=None)

## all qs 100

In [571]:
wp = 'https://www.compassedu.hk/university_0_1_t1p2p3'
contents = requests.get(wp)
soup = BeautifulSoup(contents.text)

In [584]:
qs_cn_names = []
qs_en_names = []
qs_rankings = []

In [585]:
for res in soup.find_all('div', attrs='result-list')[0].find_all('div',attrs='slot_mid'):
    # print(res)
    qs_cn_names.append(res.find('div', 'cname').contents[0])
    qs_en_names.append(res.find('div', 'ename line-1').contents[0])

In [586]:
for res in soup.find_all('div', attrs='result-list')[0].find_all('div',attrs='slot_right'):
    qs_rankings.append(res.find('div', 'sort').contents[0])

In [588]:
wp1 = 'https://www.compassedu.hk/university_0_2_t1p2p3'
contents = requests.get(wp1)
soup = BeautifulSoup(contents.text)

In [589]:
for res in soup.find_all('div', attrs='result-list')[0].find_all('div',attrs='slot_mid'):
    # print(res)
    qs_cn_names.append(res.find('div', 'cname').contents[0])
    qs_en_names.append(res.find('div', 'ename line-1').contents[0])

In [590]:
for res in soup.find_all('div', attrs='result-list')[0].find_all('div',attrs='slot_right'):
    qs_rankings.append(res.find('div', 'sort').contents[0])

In [592]:
len(qs_cn_names) 

58

In [593]:
len(qs_rankings)

58

In [594]:
qs_100_df = pd.DataFrame([qs_rankings, qs_cn_names, qs_en_names])

In [596]:
qs_100_df = qs_100_df.T

In [598]:
qs_100_df.columns = ['qs_ranking', 'ch_name', 'en_name']

In [599]:
qs_100_df.to_csv('qs_ranking_top_100.csv', index=None)